In [ ]:
#import scanpy_helpers as sh

In [ ]:
# Python packages
#import scanpy as sc
import scvi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#import  decoupler as dc

In [ ]:
import decoupler as dc

In [ ]:
path = "/data/scratch/kvalem/projects/2021/microbial-metabolites/preprocessing/40_single-cell-sorted-cd8/50_scrna_seq_by_sample/test/work/e7/055b59d0a151a805841cf4d86020b8"

In [ ]:
path_merged = "/data/scratch/kvalem/projects/2021/microbial-metabolites/preprocessing/40_single-cell-sorted-cd8/50_scrna_seq_by_sample/test/work/e7/055b59d0a151a805841cf4d86020b8/merged_adata.h5ad"

In [ ]:
path_qc = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/results_by_sample/quality_control/10mix1_adata_filtered_denoised_adata_after_qc.h5ad"

In [ ]:
path_filtered_1st_Step = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/results_by_sample/filter_sample_adata/GF-ICI2-plus_adata_filtered.h5ad"

In [ ]:
og_sample = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/scrna_seq_data/samples/adata_10mix1.h5ad"

In [ ]:
adata_solo = sc.read_h5ad(f"{path}/adata_nodoublet.h5ad")

In [ ]:
adata_merged = sc.read_h5ad(path_merged)

In [ ]:
import scanpy as sc

In [ ]:
import anndata as ad

In [ ]:
adata_solo

In [ ]:
adata_merged

In [ ]:
sc.pp.normalize_total(adata_solo)
sc.pp.log1p(adata_solo)
sc.tl.pca(adata_solo)

In [ ]:
sc.pp.neighbors(adata_solo, key_added="neighbors_uncorrected")
sc.tl.umap(adata_solo, neighbors_key="neighbors_uncorrected")

In [ ]:
adata_solo

In [ ]:
sc.pl.umap(adata_solo, color = "leiden")

In [ ]:
sc.pl.umap(adata_solo, color = "sample_id")

In [ ]:
sc.pl.umap(adata_solo, color = "condition")

In [ ]:
sc.pl.umap(adata_solo, color = "ici_therapy")

In [ ]:
sc.pl.umap(adata_solo, color = "origin")

In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF"])], color=tex_genes)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1"])], color=tex_genes)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix1"])], color=tex_genes)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF1"])], color=tex_genes)


In [ ]:
genes_in_adata = ["CD103","CD44","CD69","1Ab","2Ab"]

In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF"])], color=genes_in_adata)

In [ ]:
adata_solo.var["query"] = adata_solo.var.index
adata_solo.var

In [ ]:
list_var = adata_solo.var.index.unique()

In [ ]:
len(list_var)

In [ ]:
prefixes = "ENSMUSG"
newlist = [x for x in list_var if  x.startswith(prefixes)]

In [ ]:
import mygene

In [ ]:
mg = mygene.MyGeneInfo()
ginfo = mg.querymany(list_var, scopes='ensembl.gene')

In [ ]:
df = pd.DataFrame(ginfo)

In [ ]:
df.drop_duplicates(subset='query', keep='last', inplace=True)

In [ ]:
df

In [ ]:
df['symbol'] = df.apply(lambda row: row['query'] if pd.isna(row['symbol']) else row['symbol'], axis=1)


In [ ]:
adata_solo.var

In [ ]:
df = df.set_index('query')

In [ ]:
df

In [ ]:
adata_solo.var["query"] =adata_solo.var.index

In [ ]:
adata_solo.var
adata_solo.var.set_index('query')

In [ ]:
df2

In [ ]:
merged_df

In [ ]:
df2 = df.join(adata_solo.var, lsuffix='query', rsuffix='symbol')

#df3  = adata_solo.var.set_index('query').join(df.set_index('query'))

In [ ]:
adata_solo.var = df.merge(adata_solo.var, on='query', how='left')

In [ ]:
adata_solo.var['symbol'] = merged_df.apply(lambda row: row['query'] if pd.isna(row['symbol']) else row['symbol'], axis=1)


In [ ]:
adata_solo.var

In [ ]:
tex_genes = ["PD1"]

CXCL13 - ENSG00000156234
GZMB - ENSG00000100453
HAVCR2 - ENSG00000135077
KRT86 - ENSG00000170442    

- Tex - Exhausted T cell → CXCL13, GZMB, HAVCR2, KRT86
- Tisg - Interferon (IFN) response T cell → Isg15, Ifi44l,Ifit3, Ifit1
- Tn - Naive T cells  →Ltb, Sell,  Lef1, Nosip
- Trm - Resident Memory T cells → Itga1, Cpne7, Txnip, Il7r
- Tsen - Senescent - like T cells  → Plcg2, Igkc, Klrk1
- Tstr - Stress response T cell → Hspa1b, Hspa1a,Dnajb1, Hsph1


In [ ]:
dataframe = pd.DataFrame()

In [ ]:
# marker genes 
marker_genes= pd.read_csv("/data/scratch/kvalem/projects/2021/microbial-metabolites/preprocessing/40_single-cell-sorted-cd8/40_scrna_seq/bin/notebooks/gene_markers.csv")

In [ ]:
!pip install decoupler

In [ ]:
dc.run_ora(
    mat=adata_solo,
    net=marker_genes[["cell_type", "genesymbol"]],
    source='cell_type',
    target='genesymbol',
    min_n=1,
    verbose=True
)

In [ ]:
adata_solo.obs.sample_id.unique()

In [ ]:
adata_solo.obs.groupby(["sample_id","batch_id"]).sum()

In [ ]:
sc.tl.leiden(adata_solo, key_added="leiden", resolution=0.3)

In [ ]:
adata_list = []
for adata_file in file_list:
    adata = sc.read_h5ad(f"{path_adata_denoised_after_qc}/{adata_file}")
    adata_list.append(adata)

    # Concatenate the AnnData objects along the rows (observations)
merged_adata = ad.concat(adata_list, index_unique="-", join="outer", fill_value=0)


In [ ]:
#list of all files in the directory
import os
file_list = os.listdir("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/results_by_sample/quality_control")


In [ ]:
merged_adata = ad.concat(adata_list, index_unique="-", join="outer", fill_value=0)

In [ ]:
sc.pp.highly_variable_genes(
    merged_adata,
    layer="counts",
    n_top_genes=4000,
    subset=False,
    flavor="seurat_v3",
    batch_key="sample_id"
)


In [ ]:
adata_solo.obs.sample_id.unique()

In [ ]:
def update_columns(row):
   
    if "ICI" not in row['sample_id']:
        row['origin'] = "colon"
    else:
         row['origin'] = "til"
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
   
    if "ICI1"  in row['sample_id']:
        row['batch_id'] = "ICI1"
    elif "ICI2" in row['sample_id']:
         row['batch_id'] = "ICI2"
    elif row['sample_id'] == "10mix1":
        row['batch_id'] = "1"
    elif row['sample_id'] == "10mix2":
         row['batch_id'] = "2"
    elif row['sample_id'] == "11mix1":
        row['batch_id'] = "1"
    elif row['sample_id'] == "11mix2":
         row['batch_id'] = "2"
    elif row['sample_id'] == "GF1":
        row['batch_id'] = "1"
  
        
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
   
    if "10mix-ICI1"  in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "10mix-ICI2" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "11mix-ICI1" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "11mix-ICI2" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "GF-ICI2-plus" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "GF-ICI1-plus" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "GF-ICI1" in row['sample_id']:
        row['ici_therapy'] = "No"
    elif "GF-ICI2" in row['sample_id']:
        row['ici_therapy'] = "No"

  
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
   
    if row['sample_id'] == "10mix1":
        row['condition'] = "10mix"
    elif row['sample_id'] == "10mix2":
         row['condition'] = "10mix"
    elif row['sample_id'] == "11mix1":
         row['condition'] = "11mix"
    elif row['sample_id'] == "11mix2":
         row['condition'] = "11mix"
    elif row['sample_id'] == "GF1":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF2":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF-ICI2-plus":
         row['condition'] = "GF-plus"
    elif row['sample_id'] == "GF-ICI1-plus":
         row['condition'] = "GF-plus"
    elif row['sample_id'] == "GF-ICI2":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF-ICI1":
         row['condition'] = "GF"
    elif row['sample_id'] == "10mix-ICI1":
         row['condition'] = "10mix"
    elif row['sample_id'] == "10mix-ICI2":
         row['condition'] = "10mix"
    elif row['sample_id'] == "11mix-ICI1":
         row['condition'] = "11mix"
    elif row['sample_id'] == "11mix-ICI2":
         row['condition'] = "11mix"
    
    
        
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
adata_solo.obs[["batch_id","sample_id","origin","condition","ici_therapy"]].head(50000)

In [ ]:
adata_solo.obs.sample_id.value_counts()

In [ ]:
adata_solo.obs

In [ ]:
adata_solo.obs.origin.value_counts()

In [ ]:
# Apply the split function and expand into new columns
def split_by_last_number(text):
    match = re.search(pattern, text)
    if match:
        return match.group(1), match.group(2)
    else:
        return text, None


adata_solo.obs[['condition', 'batch_id']] = adata_solo.obs['sample_id'].apply(split_by_last_number).apply(pd.Series)


In [ ]:
def update_columns(row):
    if "10mix1" in row['sample_id']:
        row['COND'] = "10mix"
        row['batch_id'] = "1"
    elif "10mix2" in row['sample_id']:
        row['COND'] = "10mix"
        row['batch_id'] = "2"
    elif "11mix1" in row['sample_id']:
        row['COND'] = "11mix"
        row['batch_id'] = "1"
    elif "11mix2" in row['sample_id']:
        row['COND'] = "11mix"
        row['batch_id'] = "2"
    elif "GF1" in row['sample_id']:
        row['COND'] = "GF"
        row['batch_id'] = "1"
    elif "GF2" in row['sample_id']:
        row['COND'] = "GF"
        row['batch_id'] = "2"
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
    if "10mix-ICI2" in row['sample_id']:
        row['COND'] = "10mix"
        row['batch_id'] = "ICI2"
    elif "10mix-ICI1" in row['sample_id']:
        row['COND'] = "10mix"
        row['batch_id'] = "ICI1"
    elif "11mix-ICI1" in row['sample_id']:
        row['COND'] = "11mix"
        row['batch_id'] = "ICI1"
    elif "11mix-ICI2" in row['sample_id']:
        row['COND'] = "11mix"
        row['batch_id'] = "ICI2"
    elif "GF-ICI1" in row['sample_id']:
        row['COND'] = "GF"
        row['batch_id'] = "ICI1"
    elif "GF-ICI2" in row['sample_id']:
        row['COND'] = "GF"
        row['batch_id'] = "ICI2"
    elif "GF-ICI1-plus " in row['sample_id']:
        row['COND'] = "GF-plus"
        row['batch_id'] = "ICI1"
    elif "GF-ICI2-plus " in row['sample_id']:
        row['COND'] = "GF-plus"
        row['batch_id'] = "ICI2"
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
adata_solo.obs.COND.value_counts()

In [ ]:
import pandas as pd

In [ ]:
# marker genes 
marker_genes_wang_et_al= pd.read_csv("/data/scratch/kvalem/projects/2021/microbial-metabolites/tables/40_single-cell-sorted-cd8/marker_genes_cd8t_cells.csv")

In [ ]:
marker_genes_wang_et_al

In [ ]:
def sentence_case(text):
    return text.capitalize()

In [ ]:
marker_genes_wang_et_al['gene'] = marker_genes_wang_et_al['gene'].apply(sentence_case)


In [ ]:
marker_genes_wang_et_al

In [ ]:
split_columns = marker_genes_wang_et_al['cluster'].str.split('_', expand=True)
split_columns.columns = ['CD8', 'cluster', 'cell','genes']
# Concatenate the split columns with the original DataFrame
marker_genes_wang_et_al = pd.concat([marker_genes_wang_et_al, split_columns], axis=1)

In [ ]:
marker_genes_wang_et_al

In [ ]:
marker_genes_wang_et_al.rename(columns={"gene":"genesymbol","cell":"cell_type"}, inplace = True)

In [ ]:
marker_genes_wang_et_al

marker_genes_wang_et_al_unique = marker_genes_wang_et_al.drop_duplicates(subset=['genesymbol'])

In [ ]:
genes_in_adata = adata_solo[:, adata_solo.var_names.isin(marker_genes_wang_et_al_unique["genesymbol"])].var.gene_name.to_list()

In [ ]:
len(genes_in_adata)

In [ ]:
adata_solo.var

In [ ]:
marker_genes_wang_et_al_unique[["cell_type", "genesymbol"]]

In [ ]:
dc.run_ora(
    mat=adata_solo,
    net=marker_genes_wang_et_al_unique[["cell_type", "genesymbol"]],
    source='cell_type',
    target='genesymbol',
    min_n=1,
    verbose=True
)

In [ ]:
acts = dc.get_acts(adata_solo, obsm_key='ora_estimate')

In [ ]:
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

In [ ]:
acts

In [ ]:
acts.write('acts.h5ad')

In [ ]:


# We need to remove inf and set them to the maximum value observed


# We can scale the obtained activities for better visualizations
sc.pp.scale(acts)
acts

In [ ]:
marker_genes_wang_et_al_unique.cell_type.value_counts()

In [ ]:
acts.var

In [ ]:
acts.obs

## Cell types 

* Teff    - Effector T (Teff) cells 
* t-Teff  -  transitional effector T (Teff) cells 
* Tn      - Naive T cells
* Tex     - exhausted CD8+ T cell 
* Tisg    -  interferon (IFN) response  T cell 
* Tstr    - stress response T cell 
(identified a unique stress response state, TSTR, characterized by heat shock gene expression. TSTR cells are detectable in situ in the tumor microenvironment across various cancer types,)
* pTex     - precursor exhausted
* Trm      - Resident Memory T cells 
* Tcm      - T CM, central memory T cells;
* Tsen     -  senescent-like T cells 

In [ ]:
sc.pl.umap(acts, color=['Tcm','Teff','Tex','Tisg','Tn','Trm','Tsen','Tstr','pTex','sample_id', ], cmap='RdBu_r', vcenter=0)


In [ ]:
sample_list = list(set(adata_solo.obs.sample_id))

samples_in_adata = adata_solo[:, adata_solo.obs.sample_id.isin(sample_list)]

In [ ]:
sc.pl.umap(
    adata_solo[adata_solo.obs["condition"].isin(["10mix"])],
    color=["pct_counts_mt", "n_genes_by_counts"],
    cmap="inferno",
)

In [ ]:
chemokine_receptor_genes = ["CCR4","CCR5","CCR7","CXCR3","CXCR4","CXCR5","CXCR6","CCL3","CCL4","CCL5","CXCL8","XCL1","XCL2"]
chemokine_receptor_genes_sentence_case = [item.capitalize() for item in chemokine_receptor_genes]


In [ ]:
chemokine_receptor_genes_sentence_case

In [ ]:
genes_in_adata = adata_solo[:, adata_solo.var_names.isin(chemokine_receptor_genes_sentence_case)].var.gene_name.to_list()

In [ ]:
adata_solo.obs["condition"].value_counts()

In [ ]:
adata_solo.obs.columns

In [ ]:
################## CHEMOKINES
import scanpy as sc

In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["origin"].isin(["til"])], color="Cxcr3")


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["origin"].isin(["colon"])], color="Ccr4")


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF-ICI1-plus","GF-ICI2-plus"])], color="Cxcr5", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix1"])], color="Ccl5")


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix1"])], color="Ccr4")


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix2"])], color="Ccl5")


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1"])], color="Ccl5")


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix2"])], color="Ccl5")


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF"])], color=genes_in_adata)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["10mix"])], color=genes_in_adata)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["11mix"])], color=genes_in_adata)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["origin"].isin(["til"])], color=genes_in_adata)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["origin"].isin(["colon"])], color=genes_in_adata)


In [ ]:
################## GENES TAKESHI

In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF"])], color=["Klrk1", "Ccl4", "Klrc1", "Gzma", "Dapk2", "Gzmk", "Gpr55","Ifng","Mcm5","Ifit3","Ramp3","Cd8b1","Cd8a","Cd69","Cd44"])


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF-ICI1-plus","GF-ICI2-plus"])], color="Klrk1", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF-ICI1","GF-ICI2"])], color="Klrk1", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix-ICI1","10mix-ICI2"])], color="Klrk1", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix-ICI1","11mix-ICI2"])], color="Klrk1", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1","11mix2"])], color="Klrk1", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["10mix1","10mix2"])], color="Klrk1", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF1","GF2"])], color="Klrk1", frameon=False)


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF"])], color=["Cd44"])


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["10mix"])], color=["Klrk1", "Ccl4", "Klrc1", "Gzma", "Dapk2", "Gzmk", "Gpr55","Ifng","Mcm5","Ifit3","Ramp3","Cd8b1","Cd8a","Cd69","Cd44"])


In [ ]:
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["11mix"])], color=["Klrk1", "Ccl4", "Klrc1", "Gzma", "Dapk2", "Gzmk", "Gpr55","Ifng","Mcm5","Ifit3","Ramp3","Cd8b1","Cd8a","Cd69","Cd44"])


In [ ]:
sc.pl.umap(adata_solo, color=genes_in_adata)


In [ ]:
sc.pl.umap(adata_solo, color=["Klrk1", "Ccl4", "Klrc1", "Gzma", "Dapk2", "Gzmk", "Gpr55","Ifng","Mcm5","Ifit3","Ramp3"])


In [ ]:
df = dc.rank_sources_groups(acts, groupby='leiden', reference='rest', method='t-test_overestim_var')
df

In [ ]:
n_ctypes = 3
ctypes_dict = df.groupby('group').head(n_ctypes).groupby('group')['names'].apply(lambda x: list(x)).to_dict()
ctypes_dict

In [ ]:
sc.pl.matrixplot(acts, ctypes_dict, 'leiden', dendrogram=True,
                 colorbar_title='Z-scaled scores', vmin=-2, vmax=2, cmap='RdBu_r')

In [ ]:
sc.pl.violin(acts, keys=['Tex', 'Tn','Tisg'], groupby='leiden')

In [ ]:
sc.pl.violin(acts, keys=['Tsen', 'Tstr','Trm'], groupby='leiden')

In [ ]:
annotation_dict = df.groupby('group').head(1).set_index('group')['names'].to_dict()
annotation_dict


In [ ]:
annotation_dict_manual = {'0': 'T1',
 '1': 'T3',
 '2': 'T1',
 '3': 'T2',
 '4': 'T3',
 '5': 'T2',
 '6': 'T3'}

In [ ]:
adata_solo

In [ ]:
adata_solo.obs.cell_type.value_counts()

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo, color=["leiden"], wspace=1, legend_loc="on data", ax=axs , frameon=False, title="Colon & TIL integration")
plt.show()
fig.savefig('clusters_numbers.png')
fig.savefig('clusters_numbers.pdf')
fig.savefig('clusters_numbers.svg')

In [ ]:
# Add cell type column based on annotation
adata_solo.obs['cell_type'] = [annotation_dict_manual[clust] for clust in adata_solo.obs['leiden']]

# Visualize
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo, color='cell_type',legend_loc="on data", ax=axs, frameon=False,title="Colon & TIL integration")
plt.show()
fig.savefig('clusters_coarse.png')
fig.savefig('clusters_coarse.pdf')
fig.savefig('clusters_coarse.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo, color=["leiden"], wspace=1, legend_loc="on data", ax=axs , frameon=False)
plt.show()
fig.savefig('fig.png')

In [ ]:
# Add cell type column based on annotation
adata_solo.obs['cell_type'] = [annotation_dict[clust] for clust in adata_solo.obs['leiden']]

# Visualize
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo, color='cell_type',legend_loc="on data", ax=axs, frameon=False,title="Colon & TIL integration")
plt.show()
fig.savefig('clusters_cell_type.png')
fig.savefig('clusters_cell_type.pdf')
fig.savefig('clusters_cell_type.svg')

In [ ]:
# Add cell type column based on annotation
adata_solo.obs['cell_type'] = [annotation_dict[clust] for clust in adata_solo.obs['leiden']]

# Visualize
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF1","GF2"])], color='cell_type',legend_loc="on data", ax=axs, frameon=False,title="Colon & TIL integration")

plt.show()
#fig.savefig('clusters_cell_type_GF.png')
#fig.savefig('clusters_cell_type_GF.pdf')
#fig.savefig('clusters_cell_type_GF.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF1","GF2"])], color='origin',frameon=False,title="Colon & TIL integration", ax=axs)
plt.show()
#fig.savefig('clusters_origin.png')
#fig.savefig('clusters_origin.pdf')
#fig.savefig('clusters_origin.svg')

In [ ]:
# Add cell type column based on annotation
adata_solo.obs['cell_type'] = [annotation_dict[clust] for clust in adata_solo.obs['leiden']]

# Visualize
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF"])], color='cell_type',legend_loc="on data", ax=axs, frameon=False,title="Colon & TIL integration")

plt.show()
#fig.savefig('clusters_cell_type_GF.png')
#fig.savefig('clusters_cell_type_GF.pdf')
#fig.savefig('clusters_cell_type_GF.svg')

In [ ]:
# Add cell type column based on annotation
adata_solo.obs['cell_type'] = [annotation_dict[clust] for clust in adata_solo.obs['leiden']]

# Visualize
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF-plus"])], color='cell_type',legend_loc="on data", ax=axs, frameon=False,title="Colon & TIL integration")

plt.show()
#fig.savefig('clusters_cell_type_GF.png')
#fig.savefig('clusters_cell_type_GF.pdf')
#fig.savefig('clusters_cell_type_GF.svg')

In [ ]:
# Add cell type column based on annotation
adata_solo.obs['cell_type'] = [annotation_dict[clust] for clust in adata_solo.obs['leiden']]

# Visualize
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["11mix"])], color='cell_type',legend_loc="on data", ax=axs, frameon=False,title="Colon & TIL integration")

plt.show()
#fig.savefig('clusters_cell_type_GF.png')
#fig.savefig('clusters_cell_type_GF.pdf')
#fig.savefig('clusters_cell_type_GF.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF"])], color='origin',frameon=False,title="Colon & TIL integration", ax=axs)
plt.show()
#fig.savefig('clusters_origin.png')
#fig.savefig('clusters_origin.pdf')
#fig.savefig('clusters_origin.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["GF-plus"])], color='origin',frameon=False,title="Colon & TIL integration", ax=axs)
plt.show()
#fig.savefig('clusters_origin.png')
#fig.savefig('clusters_origin.pdf')
#fig.savefig('clusters_origin.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["10mix"])], color='origin',frameon=False,title="Colon & TIL integration", ax=axs)
plt.show()
#fig.savefig('clusters_origin.png')
#fig.savefig('clusters_origin.pdf')
#fig.savefig('clusters_origin.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["condition"].isin(["11mix"])], color='origin',frameon=False,title="Colon & TIL integration", ax=axs)
plt.show()
#fig.savefig('clusters_origin.png')
#fig.savefig('clusters_origin.pdf')
#fig.savefig('clusters_origin.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo, color='origin',frameon=False,title="Colon & TIL integration", ax=axs)
plt.show()
fig.savefig('clusters_origin.png')
fig.savefig('clusters_origin.pdf')
fig.savefig('clusters_origin.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo, color='condition',frameon=False,title="Colon & TIL integration", ax=axs)
plt.show()
fig.savefig('clusters_condition.png')
fig.savefig('clusters_condition.pdf')
fig.savefig('clusters_condition.svg')

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(adata_solo, color='sample_id',frameon=False,title="Colon & TIL integration", ax=axs)
plt.show()
fig.savefig('clusters_sample_id.png')
fig.savefig('clusters_sample_id.pdf')
fig.savefig('clusters_sample_id.svg')

In [ ]:
adata_solo.obs

In [ ]:
adata_solo.obs.groupby(["sample_id","cell_type"]).count()

In [ ]:
import pandas as pd

# Assuming adata_solo.obs.cell_type.value_counts() generates the cell type counts

# Get the cell type value counts
cell_type_counts = adata_solo.obs.sample_id.value_counts()

# Create a DataFrame from the cell type value counts
sample_id_df = pd.DataFrame(cell_type_counts).reset_index()

# Rename columns
sample_id_df.columns = ['sample_id', 'count']

# Display the resulting DataFrame
print(sample_id_df)


In [ ]:
sample_id_df

In [ ]:
import pandas as pd

# Assuming adata_solo.obs.cell_type.value_counts() generates the cell type counts

# Get the cell type value counts
cell_type_counts = adata_solo.obs.sample_id.value_counts()

# Create a DataFrame from the cell type value counts
cell_type_df = pd.DataFrame(cell_type_counts).reset_index()

# Rename columns
cell_type_df.columns = ['cell_type', 'count']

# Display the resulting DataFrame
print(cell_type_df)


In [ ]:
cell_type_df

In [ ]:
from matplotlib.ticker import MultipleLocator

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming cell_type_df is your DataFrame with cell type counts

# Create a bar plot using matplotlib
plt.figure(figsize=(10, 6))  # Set the figure size

# Plot the data
plt.barh(sample_id_df['sample_id'], sample_id_df['count'], fill=False)

# Customize the plot
plt.xlabel('N. of cells')
plt.ylabel('Sample')
plt.title('Number of cells per sample after filtering')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility

x_locator = MultipleLocator(base=500)
plt.gca().xaxis.set_major_locator(x_locator)

# Show the plot
plt.tight_layout()  # Adjust layout for better spacing
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming cell_type_df is your DataFrame with cell type counts

# Create a bar plot using matplotlib
plt.figure(figsize=(10, 6))  # Set the figure size

# Plot the data
plt.bar(cell_type_df['cell_type'], cell_type_df['count'], color='#b2abd2')

# Customize the plot
plt.xlabel('Cell Type')
plt.ylabel('N. of cells')
plt.title('Cell Type Counts')
plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility

# Show the plot
plt.tight_layout()  # Adjust layout for better spacing
plt.show()


In [ ]:
sc.pl.umap(
    adata_solo[adata_solo.obs["condition"].isin(["10mix"])],
    color=["pct_counts_mt", "n_genes_by_counts"],
    cmap="inferno",
)

In [ ]:
subset = adata_solo[adata_solo.obs["condition"].isin(["11mix"])]

In [ ]:
subset.obs.cell_type.value_counts()

In [ ]:
subset.obs.origin.value_counts()

In [ ]:


subset = adata_solo[adata_solo.obs["condition"].isin(["10mix"])]

In [ ]:
subset.obs.cell_type.value_counts()

In [ ]:
subset.obs.origin.value_counts()

In [ ]:


subset = adata_solo[adata_solo.obs["condition"].isin(["GF"])]

In [ ]:
subset.obs.cell_type.value_counts()

In [ ]:
subset.obs.origin.value_counts()

In [ ]:


subset = adata_solo[adata_solo.obs["condition"].isin(["GF-plus"])]

In [ ]:
subset.obs.cell_type.value_counts()

In [ ]:
subset.obs.origin.value_counts()

In [ ]:
sc.pl.umap(
    adata_solo[adata_solo.obs["cell_type"].isin(["Tex"])],
    color=["pct_counts_mt", "n_genes_by_counts"],
    cmap="inferno",
)

sc.pl.umap(adata_solo[adata_solo.obs["cell_type"].isin(["Tex"])], color=["total_counts", "leiden"], vmax=20000)

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
adata_solo.obs.sample_id.value_counts()

In [ ]:
fig, axs = plt.subplots(1,1,figsize = (5,5))
sc.pl.umap(
    adata_solo[adata_solo.obs["sample_id"].isin(["GF2"])],
    color=["pct_counts_mt"],
    cmap="inferno",
    ax = axs,
    title="pct_counts_mt GF2 Colon"
)
plt.show()
fig.savefig('GF2_pct_counts_mt.png')
fig.savefig('GF2_pct_counts_mt.pdf')
fig.savefig('GF2_pct_counts_mt.svg')

In [ ]:
fig, axs = plt.subplots(figsize = (5,5))
sc.pl.umap(
    adata_solo[adata_solo.obs["sample_id"].isin(["GF2"])],
    color=["n_genes_by_counts"],
    cmap="inferno",
    ax = axs,
    title="n_genes_by_counts GF2 Colon "
)
plt.show()
fig.savefig('GF2n_genes_by_counts.png')
fig.savefig('GF2n_genes_by_counts.pdf')
fig.savefig('GF2n_genes_by_counts.svg')

In [ ]:
fig, axs = plt.subplots(figsize = (5,5))
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF2"])], color=["total_counts"], ax = axs, vmax=20000,   title="total counts GF2 Colon ")
plt.show()
fig.savefig('GF2_total_counts.png')
fig.savefig('GF2_total_counts.pdf')
fig.savefig('GF2_total_counts.svg')


In [ ]:
fig, axs = plt.subplots(figsize = (8,8))
sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF2"])], color=["cell_type"], ax = axs, vmax=20000, title="Cell type prediction GF2 Colon ")
plt.show()
fig.savefig('GF2_cell_type.png')
fig.savefig('GF2_cell_type.pdf')
fig.savefig('GF2_cell_type.svg')


In [ ]:
sc.pl.umap(
    adata_solo[adata_solo.obs["sample_id"].isin(["GF2"])],
    color=["pct_counts_mt", "n_genes_by_counts"],
    cmap="inferno",
)

sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["GF2"])], color=["total_counts", "leiden"], vmax=20000)

In [ ]:
sc.pl.umap(adata_solo, color=["condition"], wspace=1)


In [ ]:
# The final adata filtered contains raw and log counts 
adata_solo.write_h5ad("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/adata_solo_integrated.h5ad") 

In [ ]:
adata_solo = sc.read_h5ad("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/adata_solo_integrated.h5ad")

In [ ]:
genes_heatmap = ["Klrk1", "Ccl4", "Klrc1", "Gzma", "Dapk2", "Gzmk", "Gpr55","Ifng","Mcm5","Ifit3","Ramp3"]

In [ ]:
adata_solo.raw.var_names.index = adata_solo.var.ensembl_id

In [ ]:
adata_solo

In [ ]:
adata_solo.var

In [ ]:
sc.pl.matrixplot(adata_solo, genes_heatmap, groupby="leiden", colorbar_title='Z-scaled scores', vmin=-2, vmax=2, cmap='RdBu_r')

In [ ]:
adata_solo.var.index = adata_solo.var.gene_name

In [ ]:
adata_solo.var

In [ ]:
sc.pl.heatmap(df, markers, groupby='bulk_labels', swap_axes=True)

In [ ]:
df = dc.rank_sources_groups(acts, groupby='leiden', reference='rest', method='t-test_overestim_var')
df.head()

In [ ]:
adata_solo.obs["cell_type"]

In [ ]:
sc.tl.rank_genes_groups(adata_solo, 'leiden', method='wilcoxon')

In [ ]:
sc.pp.log1p(adata_solo)

In [ ]:
adata_solo = adata_solo[~adata_solo.obs['cell_type'].isnull()]